<a href="https://colab.research.google.com/github/AnnaM-C/computational-intelligence/blob/main/GACIFAR_10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
!pip install deap
from deap import base, creator, tools

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define a simple neural network architecture
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
        x = self.fc2(x)
        return x

# Define the genetic algorithm optimizer
class CustomGAOptimizer:
    def __init__(self, mutation_rate):
        self.mutation_rate = mutation_rate

    def optimize(self, weights, biases, fitness_function):
        # Apply your GA logic to optimize weights and biases
        # Here, we'll just add some random noise to the parameters as an example
        with torch.no_grad():
            weights += self.mutation_rate * torch.randn_like(weights)
            biases += self.mutation_rate * torch.randn_like(biases)

        # Update the fitness using the updated parameters
        fitness = fitness_function(weights, biases)
        return weights, biases, fitness

# Define GA fitness function (cross-entropy loss)
def ga_fitness_function(weights, biases):
    # Create a new instance of the network with the updated weights and biases
    ga_net = SimpleClassifier(input_size=32 * 32 * 3, hidden_size=128, output_size=10)
    ga_net.fc1.weight.data = weights
    ga_net.fc1.bias.data = biases

    # Use your data to evaluate the performance of the network
    criterion = nn.CrossEntropyLoss()

    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = ga_net(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_samples += labels.size(0)

    average_loss = total_loss / total_samples
    return -average_loss  # Minimize negative of average cross-entropy loss


# Train the neural network with GA optimization
def train_with_ga_optimization(net, ga_optimizer, fitness_function, num_generations):
    for generation in range(num_generations):
        # Extract weights and biases from the first layer
        weights = net.fc1.weight
        biases = net.fc1.bias

        # Use the GA optimizer to update weights and biases
        weights, biases, fitness = ga_optimizer.optimize(weights, biases, fitness_function)

        # Update the first layer with the optimized weights and biases
        net.fc1.weight.data = weights
        net.fc1.bias.data = biases

        print(f"Generation {generation + 1}, Fitness (Negative Cross-Entropy Loss): {-fitness}")

# Example usage
input_size = 32 * 32 * 3  # CIFAR-10 images are 32x32 pixels with 3 color channels
hidden_size = 128
output_size = 10  # CIFAR-10 has 10 classes

# Create the neural network
net = SimpleClassifier(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

# Freeze all layers except the first one
for param in net.parameters():
    param.requires_grad = False
net.fc1.weight.requires_grad = True
net.fc1.bias.requires_grad = True

# Create the GA optimizer
ga_optimizer = CustomGAOptimizer(mutation_rate=0.1)

# Train the neural network with GA optimization
train_with_ga_optimization(net, ga_optimizer, ga_fitness_function, num_generations=10)


Files already downloaded and verified
Generation 1, Fitness (Negative Cross-Entropy Loss): 0.04256194557666779
Generation 2, Fitness (Negative Cross-Entropy Loss): 0.05313288961410523
Generation 3, Fitness (Negative Cross-Entropy Loss): 0.057909524641036984
Generation 4, Fitness (Negative Cross-Entropy Loss): 0.06203735655784607
Generation 5, Fitness (Negative Cross-Entropy Loss): 0.06235153489589691
Generation 6, Fitness (Negative Cross-Entropy Loss): 0.08290112684249878
Generation 7, Fitness (Negative Cross-Entropy Loss): 0.09121596368789672
Generation 8, Fitness (Negative Cross-Entropy Loss): 0.07628520467281341
Generation 9, Fitness (Negative Cross-Entropy Loss): 0.09418681173324585
Generation 10, Fitness (Negative Cross-Entropy Loss): 0.07762706762313842
